In [139]:
import pandas as pd
train = pd.read_csv("datasets/train.csv")
test = pd.read_csv("datasets/test.csv")

In [140]:
train[train.Embarked == "S"].sort_values(["Ticket", "Fare", "PassengerId"]).tail(50)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
84,85,1,2,"Ilett, Miss. Bertha",female,17.0,0,0,SO/C 14885,10.500,NaN,S
611,612,0,3,"Jardin, Mr. Jose Neto",male,NaN,0,0,SOTON/O.Q. 3101305,7.050,NaN,S
465,466,0,3,"Goncalves, Mr. Manuel Estanslas",male,38.0,0,0,SOTON/O.Q. 3101306,7.050,NaN,S
131,132,0,3,"Coelho, Mr. Domingos Fernandeo",male,20.0,0,0,SOTON/O.Q. 3101307,7.050,NaN,S
363,364,0,3,"Asim, Mr. Adola",male,35.0,0,0,SOTON/O.Q. 3101310,7.050,NaN,S
210,211,0,3,"Ali, Mr. Ahmed",male,24.0,0,0,SOTON/O.Q. 3101311,7.050,NaN,S
784,785,0,3,"Ali, Mr. William",male,25.0,0,0,SOTON/O.Q. 3101312,7.050,NaN,S
429,430,1,3,"Pickard, Mr. Berk (Berk Trembisky)",male,32.0,0,0,SOTON/O.Q. 392078,8.050,E10,S
564,565,0,3,"Meanwell, Miss. (Marion Ogden)",female,NaN,0,0,SOTON/O.Q. 392087,8.050,NaN,S
761,762,0,3,"Nirva, Mr. Iisakki Antino Aijo",male,41.0,0,0,SOTON/O2 3101272,7.125,NaN,S


In [141]:
# 나이 성별 pclass embarked fare
train = train[-train.Age.isnull()]
train_re = train[["Survived","Age","Sex","Pclass","Embarked","Fare"]].copy()
test_re = test[["Age","Sex","Pclass","Embarked","Fare"]].copy()

In [142]:
test_re.isnull().sum()

Age         86
Sex          0
Pclass       0
Embarked     0
Fare         1
dtype: int64

In [143]:
def assignAge(value): # NA처리 추가해야 함
    if value < 13:
        return "children"
    elif value <= 18:
        return "teenager"
    else:
        return "adult"

In [144]:
train_re.Age = train_re.Age.map(lambda x : assignAge(x))
test_re.Age = test_re.Age.map(lambda x : assignAge(x))

In [145]:
train_re.head()

,Survived,Age,Sex,Pclass,Embarked,Fare
0,0,adult,male,3,S,7.2500
1,1,adult,female,1,C,71.2833
2,1,adult,female,3,S,7.9250
3,1,adult,female,1,S,53.1000
4,0,adult,male,3,S,8.0500


In [146]:
for col in train_re.columns:
    if col != "Fare":
        train_re[col] = train_re[col].astype("category")
train_re["Fare"] = train_re["Fare"].astype("float32")

for col in test_re.columns:
    if col != "Fare":
        test_re[col] = test_re[col].astype("category")
test_re["Fare"] = test_re["Fare"].astype("float32")

In [147]:
test_re.dtypes

Age         category
Sex         category
Pclass      category
Embarked    category
Fare         float32
dtype: object

In [152]:
from sklearn.linear_model import LogisticRegression
notFare = ["Age", "Pclass", "Sex", "Embarked"]
train_X = pd.get_dummies(train_re[notFare])
train_X = pd.concat([train_X, train_re["Fare"]], axis=1)
train_Y = train_re["Survived"]

logm = LogisticRegression()
logm.fit(train_X, train_Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [153]:
from sklearn.metrics import confusion_matrix, accuracy_score

pred = logm.predict(train_X)
confmat = confusion_matrix(train_Y, pred)
accuracy_score(train_Y, pred)

0.7801120448179272

In [175]:
test_re.Fare = test_re.Fare.fillna(0)

In [132]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [184]:
test_X = pd.get_dummies(test_re[notFare])
test_X = pd.concat([test_X, test_re["Fare"]], axis=1)

In [185]:
logm.predict(test_X)

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [190]:
sub = pd.read_csv("datasets/gender_submission.csv")

In [192]:
sub.Survived = logm.predict(test_X)

In [196]:
sub.to_csv("datasets/gender_submissino_logistic.csv", index=False)

In [197]:
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier()
ridge.fit(train_X, train_Y)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [199]:
accuracy_score(train_Y, ridge.predict(train_X))

0.7913165266106442

In [201]:
sub.Survived = ridge.predict(test_X)
sub.to_csv("datasets/gender_submission_ridge.csv", index=False)

In [202]:
from sklearn.linear_model import Lasso
lasso = Lasso()
lasso.fit(train_X, train_Y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [223]:
def cutoff(value):
    if value > 0.5:
        return 1
    else:
        return 0
    
pred = lasso.predict(train_X)
pred = pd.Series(pred)
pred = pred.map(lambda x : cutoff(x))

In [224]:
accuracy_score(train_Y, pred)

0.6484593837535014

In [216]:
sub.Survived = pd.Series(lasso.predict(test_X)).map(lambda x : cutoff(x))
sub.to_csv("datasets/gender_submission_lasso.csv", index=False)

In [225]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(train_X, train_Y)

accuracy_score(train_Y, rf.predict(train_X))

0.9145658263305322

In [226]:
sub.Survived = rf.predict(test_X)
sub.to_csv("datasets/gender_submission_randomforest.csv", index=False)